# Build a Causal Language Model Using PyTorch
In notebook07 we learned how to build a causal language model (GPT) from scratch. In this homework, we will build a Causal Language model with PyTorch, which also supports back-propagation compared to the notebook implementation.

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 1000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

We are going to train our GPT on this corpus

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(text[:1000])

--2023-05-09 11:04:54--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt'

input.txt           100%[===================>]   1.06M  2.72MB/s    in 0.4s    

2023-05-09 11:04:55 (2.72 MB/s) - 'input.txt' saved [1115394/1115394]

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it b

For simplicity, character-level tokenization is applied here instead of learning a new BPE tokenizer

In [3]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [30]:
#playing with the data to understand it

print(encode('hej'))
print(decode([45,34]))

get_batch('train')

[46, 43, 48]
gV


(tensor([[43,  1, 53, 44, 44, 43, 56,  8,  0,  0, 30, 27, 25, 17, 27, 10,  0, 14,
          47, 42,  1, 46, 43, 56,  1, 42, 43, 60, 47, 57, 43,  0],
         [58, 59, 56, 43,  2,  0, 15, 46, 47, 42, 43,  1, 51, 43,  6,  1, 42, 43,
          39, 56,  1, 57, 58, 53, 52, 43,  6,  1, 58, 46, 39, 58],
         [ 1, 14, 59, 41, 49, 47, 52, 45, 46, 39, 51,  1, 42, 53, 58, 46,  1, 58,
          59, 56, 52,  1, 46, 47, 57,  1, 46, 39, 58, 43,  0, 27],
         [ 1, 51, 39, 52, 63,  1, 51, 53, 59, 58, 46, 57,  7,  7,  0, 20, 53, 61,
           1, 54, 56, 53, 40, 39, 40, 50, 43,  1, 21,  1, 42, 53],
         [63, 53, 59,  8,  0,  0, 13, 26, 19, 17, 24, 27, 10,  0,  5, 32, 47, 57,
           1, 53, 52, 43,  1, 58, 46, 47, 52, 45,  1, 58, 53,  1],
         [39, 49, 10,  0, 32, 46, 39, 58,  1, 13, 52, 45, 43, 50, 53,  5, 57,  1,
          44, 53, 56, 57, 61, 53, 56, 52, 11,  1, 47, 57,  1, 47],
         [60, 43,  1, 41, 56, 39, 64, 43, 42,  1, 51, 63,  1, 60, 53, 47, 41, 43,
           6,  0, 32, 46

In [33]:
get_batch('train')[1].shape

torch.Size([16, 32])

In [124]:
#playing with torch

m = torch.randn(4,3)
print(m)

torch.transpose(m, 1, 0)

tensor([[-2.3069,  1.2809,  0.4196],
        [-0.1880, -0.1550,  0.9203],
        [ 0.1749, -1.7687,  0.3431],
        [ 1.9464,  0.3710,  1.0410]])


tensor([[-2.3069, -0.1880,  0.1749,  1.9464],
        [ 1.2809, -0.1550, -1.7687,  0.3710],
        [ 0.4196,  0.9203,  0.3431,  1.0410]])

### TODO1: Single-head Attention Implementation

In [125]:
n = torch.matmul(m, torch.transpose(m, 0, 1))
n

tensor([[ 7.1386,  0.6213, -2.5250, -3.5780],
        [ 0.6213,  0.9063,  0.5571,  0.5346],
        [-2.5250,  0.5571,  3.2766,  0.0413],
        [-3.5780,  0.5346,  0.0413,  5.0097]])

In [130]:
#trying out mask

n.masked_fill(torch.tril(torch.ones(4,4))==0,float('-inf'))

np.sqrt(m.shape[-1])

print(torch.tril(torch.ones(block_size, block_size)).shape)




torch.Size([32, 32])

In [221]:
#intuition in 3d matrix multiplication

a = np.random.randint(0, 10, size=(3, 6, 3))
b = np.random.randint(0, 10, size=(3, 3, 4))
print((a@b).shape)
print(np.transpose(a,(0,2,1)).shape)

#trying torch
print(torch.from_numpy(a).shape)
torch.from_numpy(a).transpose(-2,-1).shape

(3, 6, 4)
(3, 3, 6)
torch.Size([3, 6, 3])


torch.Size([3, 3, 6])

In [211]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False) #head_size is later set to 16
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # register_buffer simply adds self.tril as a non-paramter buffer in this nn.Module
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        #example from notebook -> softmax(q @ k.T / np.sqrt(q.shape[-1])) @ v
        # calculate query, key, and value
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)
        
        #print('x shape: ', x.shape) -> torch.Size([16, 32, 64])
        #print('qkv -shapes: ', q.shape, k.shape, v.shape) -> torch.Size([16, 32, 16])

        # calculate attention scores
        scores = torch.matmul(q, k.transpose(-2, -1)) / np.sqrt(q.shape[-1])
        
        #print('score_1 : ', scores.shape) #-> torch.Size([16, 32, 32])  
        
        scores = self.softmax(scores)

        # apply dropout
        scores = self.dropout(scores)
        
        #print('score_2 : ', scores.shape) -> torch.Size([16, 32, 32])

        # apply mask
        output = scores.masked_fill(self.tril == 0, float('-inf'))
        
        #final attention
        output = torch.matmul(output, v)

        # return output
        return output
    
    # TODO: fill in the forward pass for single-head attention
    # hint1 can use self.tril
    # hint2 can use masked_fill
    # Todo hint3 concatinate heads somehow --> getting only nan...

### Multi-head attention and feed forward layer implementations

In [212]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        #USING Head() here
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

### TODO2: Transformer Block Implementation

In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # TODO: fill in the forward pass
        #attention
        sa = self.sa(x)
        x = self.ln1(x + sa)
        
        #MLP
        ffwd = self.ffwd(x)
        x = self.ln2(x + ffwd)
        
        # Hint: don't forget the residual connections
        return x

### Language Model Implementation.

In [217]:
class LanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        #USING block here
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [218]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

model = LanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss nan, val loss nan
step 100: train loss nan, val loss nan
step 200: train loss nan, val loss nan
step 300: train loss nan, val loss nan
step 400: train loss nan, val loss nan
step 500: train loss nan, val loss nan


KeyboardInterrupt: 